In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.graphics.api as smg
import statsmodels.formula.api as smf

sns.set_style('darkgrid')

In [2]:
def linearity_test(model, y):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    It uses Locally Weighted Scatterplot Smoothing (LOWESS) to fit a model. 
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    pred = model.fittedvalues
    influence = model.get_influence()
    resid_std = influence.resid_studentized_internal
    
    fig, ax = plt.subplots(1,2, figsize=(7.5,3.5))
    
    sns.regplot(x=pred, y=y, lowess=True, ax=ax[0], line_kws={'color':'darkorchid'})
    # I've added the ideal line (y=yhat) for comparison
    sns.lineplot(x=[min(pred), max(pred)], y=[min(pred), max(pred)], 
                 ax=ax[0], color='red', ls=':')
    ax[0].set_title('Observed vs. Fitted Values')
    ax[0].set_xlabel('Fitted')
    ax[0].set_ylabel('Observed')
    
    sns.regplot(x=pred, y=resid_std, lowess=True, ax=ax[1], line_kws={'color':'darkorchid'})
    # I've added the ideal line (y=0) for comparison
    sns.lineplot(x=[min(pred), max(pred)], y=[0,0], ax=ax[1], color='red', ls=':')
    ax[1].set_title('Residuals vs. Fitted Values')
    ax[1].set_xlabel('Fitted')
    ax[1].set_ylabel('Standardized Residual')
    
    return fig, ax

In [3]:
def homoscedasticity_test(model):
    '''
    Function for testing the homoscedasticity of residuals in a linear regression model.
    It plots residuals and standardized residuals vs. fitted values and runs Breusch-Pagan 
    and Goldfeld-Quandt tests.
    
    Args:
    * model - fitted OLS model from statsmodels
    '''
    import statsmodels.stats.api as sms
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    pred = model.fittedvalues
    resid = model.resid
    resid_z = model.get_influence().resid_studentized_internal

    fig, ax = plt.subplots(1,2, figsize=(12,5.5))

    sns.regplot(x=pred, y=resid, lowess=True, ax=ax[0], line_kws={'color': 'darkorchid'})
    # I've added the ideal line (y=0) for comparison
    sns.lineplot(x=[min(pred), max(pred)], y=[0,0], ax=ax[0], color='red', ls=':')
    ax[0].set_title('Residuals vs. Fitted Values')
    ax[0].set(xlabel='Fitted', ylabel='Residual')

    sns.regplot(x=pred, y=np.sqrt(np.abs(resid_z)), lowess=True, ax=ax[1], line_kws={'color': 'darkorchid'})
    # I've added the ideal line for comparison
    sns.lineplot(x=[min(pred), max(pred)], y=[0.822,0.822], ax=ax[1], color='red', ls=':')
    ax[1].set_title('Scale-Location')
    ax[1].set(xlabel='Fitted', ylabel=r'$\sqrt{|{\mathrm{Standardized~Residual}}|}$')

    # Breusch-Pagan tests if regression on 'Residuals ~ Fitted Values' has non-zero slope
    # Good for picking up trends of strictly increasing or decreasing variance
    bp_test = pd.DataFrame(sms.het_breuschpagan(resid, model.model.exog), 
                           columns=['value'],
                           index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])

    # Goldfeld-Quandt tests if variance on left half of residual plot is equal to variance on right half
    gq_test = pd.DataFrame(sms.het_goldfeldquandt(resid, model.model.exog)[:-1],
                           columns=['value'],
                           index=['F statistic', 'p-value'])

    print('\n Breusch-Pagan test ----')
    print(bp_test)
    print('\n Goldfeld-Quandt test ----')
    print(gq_test)
    print('\n Residuals plots ----')
    plt.show()

In [4]:
def normality_of_residuals_test(model):
    '''
    Function for drawing the normal QQ-plot of the residuals and running 5 statistical tests to 
    investigate the normality of residuals.
    
    Arg:
    * model - fitted OLS models from statsmodels
    '''
    import scipy.stats as stats
    import statsmodels.graphics.api as smg
    import statsmodels.stats.api as sms
    
    fig, ax = plt.subplots(figsize=(4.5,4))
    smg.qqplot(data=model.resid, line='45', fit=True, ax=ax);
    ax.set_title('Q-Q plot');
    resid_z = model.get_influence().resid_studentized_internal

    sw = stats.shapiro(model.resid)
    dp = stats.normaltest(model.resid)
    jb = stats.jarque_bera(model.resid)
    ad = stats.anderson(model.resid, dist='norm')
    lf = sms.lilliefors(model.resid)
    ks = stats.kstest(resid_z, 'norm')
    
    print(f'Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}')
    print(f"D'Agostino-Pearson Omnibus test ---- statistic: {dp[0]:.4f}, p-value: {dp[1]:.4f}")
    print(f'Lilliefors test ---- statistic: {lf[0]:.4f}, p-value: {lf[1]:.4f}')
    print(f'Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}')
    print(f'Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}')
    print(f'Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}')
    print('If the returned AD statistic is larger than the critical value, then for the 5% significance level, '\
          'the null hypothesis that the data come from the Normal distribution should be rejected. ')
    
    plt.show()

In [5]:
def autocorrelation_plot(model):
    import statsmodels.tsa.api as smt
    fig, ax = plt.subplots(figsize=(12,6))
    acf = smt.graphics.plot_acf(model.resid, lags=40, alpha=0.05, ax=ax)

# US Crime Data (Feature Selection Gone Mad)

In [11]:
crime = pd.read_csv('UScrime.csv')

In [12]:
crime.columns

Index(['M', 'So', 'Ed', 'Po1', 'Po2', 'LF', 'M.F', 'Pop', 'NW', 'U1', 'U2',
       'Wealth', 'Ineq', 'Prob', 'Time', 'Crime'],
      dtype='object')

Note: I renamed the M.F column in the csv, so you won't need to rename it.

In [13]:
crime.rename(columns={'M.F':'MF'},inplace=True)

In [15]:
# crime.drop('Unnamed: 0',axis=1,inplace=True)

In [16]:
X = crime.drop('Crime',axis=1)
y = crime['Crime']

## Brute Force Method
This is only feasible for smaller datasets, especially with few or no interactions under consideration.
However, it is possible if you have enough data for a full model (must have at least k+2 data points).

In [17]:
def columns_to_formulas(X, y, max_terms=None):
    '''
    Function for creating all possible formulas from the columns up to 
    the given degree (adding no interactions or higher order terms)
    
    Args:
    * X - the design matrix (DataFrame)
    * y - the response data (Series)
    * deg - the maximum degree. If deg=None, will calculate all possible interactions. By default, deg=None.
    
    Returns:
    * List of Patsy formulas covering those terms
    '''
    from itertools import chain, combinations
    if max_terms==None:
        max_terms = len(X.columns)
    formulas = []
    models = list(
        chain.from_iterable(
            combinations(X.columns,terms) for terms in range(1, max_terms+1)
        )
    )
    for model in models:
        formula = y.name + ' ~ '
        for term in model:
            formula += term + ' + '
        formulas.append(formula[:-3])
    return formulas

In [18]:
formulas = columns_to_formulas(X,y,max_terms=None)
len(formulas)

32767

In [19]:
import time

In [20]:
tic = time.process_time()

formulas = columns_to_formulas(X,y,max_terms=None)
r2 = []
r2a = []
aic = []
bic = []

for formula in formulas:
    model = smf.ols(formula,crime).fit()
    r2.append(model.rsquared)
    r2a.append(model.rsquared_adj)
    aic.append(model.aic)
    bic.append(model.bic)
results = pd.DataFrame({'Formula':formulas,'R2':r2,'R2a':r2a,'AIC':aic,'BIC':bic})

toc = time.process_time()
print(f'This took a decent laptop {toc-tic:.4f} seconds to run.')

This took a decent laptop 464.6875 seconds to run.


In [21]:
results.iloc[results['R2a'].argmax()]

Formula    Crime ~ M + Ed + Po1 + MF + U1 + U2 + Ineq + Prob
R2                                                  0.788827
R2a                                                 0.744369
AIC                                               637.315101
BIC                                                653.96643
Name: 18493, dtype: object

In [22]:
results.iloc[results['AIC'].argmin()]

Formula    Crime ~ M + Ed + Po1 + MF + U1 + U2 + Ineq + Prob
R2                                                  0.788827
R2a                                                 0.744369
AIC                                               637.315101
BIC                                                653.96643
Name: 18493, dtype: object

In [23]:
results.iloc[results['BIC'].argmin()]

Formula    Crime ~ M + Ed + Po1 + U2 + Ineq + Prob
R2                                        0.765866
R2a                                       0.730746
AIC                                      638.16613
BIC                                     651.117163
Name: 5816, dtype: object

## Select K Best
Quick but not altogether effective method for regression. Runs a regression model with all terms and takes the $k$ factors with the lowest p-values.

In [24]:
from sklearn.feature_selection import SelectKBest, f_regression

In [25]:
test = SelectKBest(score_func=f_regression, k=5)
fit = test.fit(X,y)

In [26]:
X.columns[fit.get_support()]

Index(['Po1', 'Po2', 'Pop', 'Wealth', 'Prob'], dtype='object')

In [27]:
def form(cols, y):
    '''
    Useful only here. Builds a Patsy formula from the outcome of K best.
    '''
    formula = y.name + ' ~ '
    for term in cols:
        formula += term + ' + '
    return formula[:-3]

In [36]:
for col in X.columns:
    model = smf.ols(f'Crime ~ {col}',crime).fit()
    p = model.f_pvalue
    print(f'{col:6}: {p:.4f}')

M     : 0.5498
So    : 0.5446
Ed    : 0.0269
Po1   : 0.0000
Po2   : 0.0000
LF    : 0.2036
MF    : 0.1488
Pop   : 0.0204
NW    : 0.8278
U1    : 0.7362
U2    : 0.2331
Wealth: 0.0019
Ineq  : 0.2286
Prob  : 0.0027
Time  : 0.3147


That's really all this is doing - picking the k best from that list. Note that the next one that would be added is Ed.

In [29]:
test = SelectKBest(score_func=f_regression, k=6)
fit = test.fit(X,y)
X.columns[fit.get_support()]

Index(['Ed', 'Po1', 'Po2', 'Pop', 'Wealth', 'Prob'], dtype='object')

In [30]:
tic = time.process_time()

formulas = []
r2 = []
r2a = []
aic = []
bic = []

for kk in range(1,len(X.columns)):
    fit = SelectKBest(score_func=f_regression, k=kk).fit(X,y) 
    formula = form(X.columns[fit.get_support()],y)
    model = smf.ols(formula,crime).fit()
    formulas.append(formula)
    r2.append(model.rsquared)
    r2a.append(model.rsquared_adj)
    aic.append(model.aic)
    bic.append(model.bic)
kbest_results = pd.DataFrame({'Formula':formulas,'R2':r2,'R2a':r2a,'AIC':aic,'BIC':bic})

toc = time.process_time()
print(f'This took a decent laptop {toc-tic:.4f} seconds to run.')

This took a decent laptop 0.2656 seconds to run.


In [31]:
kbest_results.iloc[kbest_results['R2a'].argmax()]

Formula    Crime ~ M + So + Ed + Po1 + Po2 + LF + MF + Po...
R2                                                  0.800413
R2a                                                 0.713094
AIC                                               646.662873
BIC                                               674.415087
Name: 13, dtype: object

In [32]:
kbest_results.iloc[kbest_results['AIC'].argmin()]

Formula    Crime ~ M + So + Ed + Po1 + Po2 + LF + MF + Po...
R2                                                  0.800413
R2a                                                 0.713094
AIC                                               646.662873
BIC                                               674.415087
Name: 13, dtype: object

In [33]:
kbest_results.iloc[kbest_results['BIC'].argmin()]

Formula    Crime ~ Ed + Po1 + Po2 + LF + MF + Pop + Wealt...
R2                                                  0.733915
R2a                                                 0.669191
AIC                                               650.178597
BIC                                               668.680074
Name: 8, dtype: object

## Recursive Feature Elimination
This is basically Stepwise Regression without relying on p-values, instead using magnitude of parameter estimates.

In [37]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [40]:
model = LinearRegression(fit_intercept=True)
rfe = RFE(model, n_features_to_select=6)
fit = rfe.fit(X,y)

In [41]:
X.columns[fit.get_support()]

Index(['So', 'Ed', 'Po1', 'Po2', 'LF', 'Prob'], dtype='object')

In [42]:
formula = form(X.columns[fit.get_support()],y)

In [43]:
smf.ols(formula, crime).fit().summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:               OLS              Adj. R-squared:     0.567   
Dependent Variable:  Crime            AIC:                660.4977
Date:                2023-08-21 17:51 BIC:                673.4487
No. Observations:    47               Log-Likelihood:     -323.25 
Df Model:            6                F-statistic:        11.04   
Df Residuals:        40               Prob (F-statistic): 3.12e-07
R-squared:           0.623            Scale:              64774.  
------------------------------------------------------------------
            Coef.     Std.Err.    t    P>|t|    [0.025     0.975] 
------------------------------------------------------------------
Intercept -1329.6954  716.5877 -1.8556 0.0709 -2777.9731  118.5823
So          413.9895  122.2126  3.3875 0.0016   166.9886  660.9903
Ed           67.2912   57.4824  1.1706 0.2487   -48.8850  183.4674
Po1         239.1621  117.3743  2.0376 0.0482     1.9398  476.3845
Po2        -168.4159  127.2207 -1.3238 0.1931  -425.5385   88.7066
LF         1658.9000 1226.2691  1.3528 0.1837  -819.4823 4137.2824
Prob      -4875.0868 2094.1173 -2.3280 0.0251 -9107.4556 -642.7179
------------------------------------------------------------------
Omnibus:               0.059        Durbin-Watson:           2.190
Prob(Omnibus):         0.971        Jarque-Bera (JB):        0.101
Skew:                  0.068        Prob(JB):                0.951
Kurtosis:              2.817        Condition No.:           915  
==================================================================

"""

In [47]:
from sklearn.preprocessing import StandardScaler
Xstd = StandardScaler().fit_transform(X)
Xstd = pd.DataFrame(data=Xstd, columns=X.columns)

In [48]:
formula = form(X.columns,y)

In [49]:
model = smf.ols(formula, Xstd.join(y)).fit()
print(model.summary2())
# print(model.summary2().tables[1].sort_values('Coef.',ascending=False,key=abs)['Coef.'])

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.708   
Dependent Variable: Crime            AIC:                648.0291
Date:               2023-08-21 17:53 BIC:                677.6314
No. Observations:   47               Log-Likelihood:     -308.01 
Df Model:           15               F-statistic:        8.429   
Df Residuals:       31               Prob (F-statistic): 3.54e-07
R-squared:          0.803            Scale:              43708.  
-----------------------------------------------------------------
              Coef.   Std.Err.    t    P>|t|    [0.025    0.975] 
-----------------------------------------------------------------
Intercept    905.0851  30.4952 29.6796 0.0000  842.8898  967.2804
M            109.2012  51.8638  2.1055 0.0434    3.4243  214.9780
So            -1.8023  70.4880 -0.0256 0.9798 -145.5634  141.9589
Ed           208.4251  68.7154  3.0332 0.0049   68.2792  348.5710
Po1          566.8662 311.9

In [50]:
formula = form(X.columns,y)
formula += ' - So'
model = smf.ols(formula, Xstd.join(y)).fit()
print(model.summary2().tables[1].sort_values('Coef.',ascending=False,key=abs)['Coef.'])

Intercept    905.085106
Po1          566.648663
Po2         -302.117718
Ineq         278.080663
Ed           208.320589
U2           139.808715
Prob        -109.401453
M            109.075166
U1          -103.203798
Wealth        91.215642
MF            50.510115
NW            42.061587
Pop          -27.612448
LF           -25.829084
Time         -24.222806
Name: Coef., dtype: float64


In [51]:
formula = form(X.columns,y)
formula += ' - So - Time - LF'
model = smf.ols(formula, Xstd.join(y)).fit()
print(model.summary2().tables[1].sort_values('Coef.',ascending=False,key=abs)['Coef.'])

Intercept    905.085106
Po1          486.720869
Ineq         280.631937
Po2         -203.387151
Ed           198.426855
U2           139.412519
M            111.029144
Prob         -90.425657
U1           -89.402928
Wealth        85.849659
MF            41.856653
Pop          -37.087781
NW            29.317738
Name: Coef., dtype: float64


In [52]:
formula = form(X.columns,y)
formula += ' - So - Time - LF - NW'
model = smf.ols(formula, Xstd.join(y)).fit()
print(model.summary2().tables[1].sort_values('Coef.',ascending=False,key=abs)['Coef.'])

Intercept    905.085106
Po1          473.961149
Ineq         292.523629
Ed           194.586853
Po2         -176.785793
U2           142.642310
M            121.300655
U1           -90.914389
Prob         -84.205101
Wealth        81.092777
MF            38.393079
Pop          -35.658671
Name: Coef., dtype: float64


In [53]:
formula = form(X.columns,y)
formula += ' - So - Time - LF - NW - Pop'
model = smf.ols(formula, Xstd.join(y)).fit()
print(model.summary2().tables[1].sort_values('Coef.',ascending=False,key=abs)['Coef.'])

Intercept    905.085106
Po1          434.462305
Ineq         274.472696
Ed           193.350563
Po2         -156.843790
U2           143.912814
M            123.796979
U1           -97.418899
Prob         -77.897614
Wealth        72.554853
MF            55.676023
Name: Coef., dtype: float64


In [55]:
# Skipping ahead to the final 6 using the same method repeatedly
formula = form(X.columns,y)
formula += ' - So - Time - LF - NW - Pop - MF - U1 - U2 - Prob'
model = smf.ols(formula, Xstd.join(y)).fit()
print(model.summary2().tables[1].sort_values('Coef.',ascending=False,key=abs)['Coef.'])

Intercept    905.085106
Po1          566.913479
Ineq         326.865102
Po2         -253.725083
Wealth       168.065292
Ed           163.811991
M            112.806354
Name: Coef., dtype: float64


## That's all for now!
You've now been exposed to three possible methods: brute force, k-best, and RFE. Hopefully you've seen that none of them are particularly well-suited to most problems. If nothing else, the brute force method should make you appreciate the difficulty of finding an algorithm that effectively sorts through all possible models in an efficient manner. 

Add additional note: in application, brute force will also almost guarantee you overfit the model; the "optimal" model in the training set is rarely effective in the validation set. If you select a model off of the validation set, you overuse that set and it is rarely effective in the test set. This is another reason we don't typically use brute force.